In [1]:
import pathlib
import pandas as pd
import dask.dataframe as dd
from config import load_config
import visualizations
import numpy as np
import utils
config = load_config()

origin_files = pathlib.Path(config['datapath_30G'])
split_file = pathlib.Path("split_file") / origin_files.name
split_file.mkdir(exist_ok=True,parents=True)

In [2]:
df = dd.read_parquet(origin_files)
df = df.repartition(npartitions=300)

In [3]:
df.to_parquet(split_file)


In [11]:
df.head(10)

,id,last_login,user_name,fullname,email,age,income,gender,country,address,purchase_history,is_active,registration_date,phone_number,login_history
0,0,2024-12-02T03:49:12+00:00,RKWKCXRZFV,瞿紫玉,kuegujsk@hotmail.com,82,366311.83,女,美国,Non-Chinese Address Placeholder,"{""avg_price"":9496,""categories"":""零食"",""items"":[{...",False,2024-10-31,+1 (804) 855-6279,"{""avg_session_duration"":105,""devices"":[""deskto..."
1,1,2024-08-25T05:39:16+00:00,RCLELJ,李泽宸,wslfszer@126.com,71,833917.30,男,英国,上海市淄博山水路360号,"{""avg_price"":3014,""categories"":""手套"",""items"":[{...",True,2023-01-13,+44 1850 116429,"{""avg_session_duration"":64,""devices"":[""mobile""..."
2,2,2023-12-21T14:28:09+00:00,KSHSK,詹紫玥,gputsgbf@126.com,54,839379.17,女,澳大利亚,北京市东莞保健中心路614号,"{""avg_price"":8921,""categories"":""裙子"",""items"":[{...",True,2022-07-06,+61 656 440 523,"{""avg_session_duration"":116,""devices"":[""deskto..."
3,3,2023-06-06T03:21:09+00:00,CCJMXPJA,姜小红,akidhwzo@outlook.com,18,383963.16,男,巴西,山东省株洲配送中心路176号,"{""avg_price"":939,""categories"":""耳机"",""items"":[{""...",False,2020-03-20,+55 54 34995-1600,"{""avg_session_duration"":25,""devices"":[""mobile""..."
4,4,2024-10-08T11:02:18+00:00,TJRJDNO,童泽楠,suupywzi@qq.com,77,337059.32,男,英国,浙江省赤峰安康路957号,"{""avg_price"":959,""categories"":""手套"",""items"":[{""...",False,2023-01-05,+44 5383 067377,"{""avg_session_duration"":51,""devices"":[""desktop..."
5,5,2024-09-04T13:44:36+00:00,AKYKTTY,韩泽潼,gixviwwg@outlook.com,53,811069.41,男,德国,甘肃省大连海洋公园路422号,"{""avg_price"":5392,""categories"":""帽子"",""items"":[{...",True,2024-04-15,+49 426 9016161,"{""avg_session_duration"":108,""devices"":[""tablet..."
6,6,2023-06-26T04:48:53+00:00,NOJMS,云泽瑜,eizxwtic@hotmail.com,39,796578.18,男,中国,Non-Chinese Address Placeholder,"{""avg_price"":738,""categories"":""水产"",""items"":[{""...",False,2022-03-19,+86 695-7090-7964,"{""avg_session_duration"":109,""devices"":[""deskto..."
7,7,2024-11-15T15:23:17+00:00,BAJEZV,花娟,ztoojhig@163.com,95,846955.79,男,日本,宁夏回族自治区温州物流中心路931号,"{""avg_price"":4194,""categories"":""智能手表"",""items"":...",True,2021-08-27,+81 98-7196-4386,"{""avg_session_duration"":114,""devices"":[""deskto..."
8,8,2024-10-05T10:00:00+00:00,LCODJG,嵺秀英,vagiglux@outlook.com,62,214963.84,女,日本,Non-Chinese Address Placeholder,"{""avg_price"":631,""categories"":""音响"",""items"":[{""...",False,2023-03-07,+81 98-0912-4108,"{""avg_session_duration"":88,""devices"":[""desktop..."
9,9,2023-10-06T20:34:22+00:00,WCBIXG,穆泽洲,fapadldt@gmail.com,60,831625.38,男,印度,西藏自治区本溪深圳路687号,"{""avg_price"":6418,""categories"":""玩具"",""items"":[{...",True,2021-07-26,+91 68822 42253,"{""avg_session_duration"":102,""devices"":[""deskto..."


In [17]:
import json

# 定义每个 partition 的处理函数
def extract_avg_price(pdf):
    # 解析 JSON 并提取 avg_price
    pdf['avg_price_extracted'] = pdf['purchase_history'].apply(
        lambda x: json.loads(x).get('avg_price', None)
    )
    return pdf[['avg_price_extracted']]

# 应用函数
df_avg = df.map_partitions(extract_avg_price)

# 转换为 float（如果需要）并计算平均值
df_avg['avg_price_extracted'] = df_avg['avg_price_extracted'].astype(float)
result = df_avg['avg_price_extracted'].mean().compute()

print("平均 avg_price：", result)


ValueError: Metadata inference failed in `extract_avg_price`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')

Traceback:
---------
  File "c:\Users\LiuYicheng\.conda\envs\data-mining\lib\site-packages\dask\dataframe\utils.py", line 149, in raise_on_meta_error
    yield
  File "c:\Users\LiuYicheng\.conda\envs\data-mining\lib\site-packages\dask\dataframe\dask_expr\_expr.py", line 4054, in emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "C:\Users\LiuYicheng\AppData\Local\Temp\ipykernel_15804\2370044332.py", line 6, in extract_avg_price
    pdf['avg_price_extracted'] = pdf['purchase_history'].apply(
  File "c:\Users\LiuYicheng\.conda\envs\data-minin